In [1]:
import time

start = time.time()

In [3]:
import pandas as pd
# intent = pd.read_csv('intent.csv')
persona = pd.read_csv('persona.csv')
subject = pd.read_csv('subject.csv')
template = pd.read_csv('template.csv')
pma = pd.read_csv('pma.csv')
match_type = {'match_type': ['br', 'xx']}
match_type = pd.DataFrame(match_type)
ad_group_name = {'ad_group_template': ['subject,intent,pma,match_type']}
ad_group_name = pd.DataFrame(ad_group_name)
location = pd.read_csv("location.csv")
market = pd.read_csv('market.csv', usecols=['language', 'market', 'account', 'account_id'])
campaign = pd.read_csv('campaign.csv', usecols=['market', 'campaign_name_template'])
other = pd.read_csv('other.csv')
medium = pd.read_csv('medium.csv')

In [3]:
intermediate = template.merge(pma, on='intent_localised', how='inner')[['intent_localised', 'keyword_template', 'language', 'pma']]
intermediate = intermediate.merge(market, on='language', how='inner')
intermediate = intermediate.merge(campaign, on='market', how='inner')
intermediate = intermediate.merge(match_type, how='cross')
intermediate = intermediate.merge(ad_group_name, how='cross')

In [4]:
intermediate.rename(columns={'intent_localised': 'intent'}, inplace=True)

In [5]:
intermediate = intermediate[:2]

In [6]:
import numpy as np
from itertools import product

dimension_dfs = {
    'subject': subject,
    'persona': persona,
    'other' : other,
    'medium': medium
}

def create_keywords(row):
    dimensions = list(row['keyword_template'].split(','))
    language = row['language']
    market = row['market']
    combinations = []

    for dimension in dimensions:

        if dimension == 'location':
            combinations.append(location['location'][location.market == market].values.tolist())
            continue

        if dimension == 'intent':
            combinations.append([row["intent"]])
            continue
        
        if dimension in dimension_dfs:
            combinations.append(dimension_dfs[dimension].loc[dimension_dfs[dimension]['language'] == language, f'{dimension}_localised'].values.tolist())
        else:
            combinations.append([dimension])

    # Create all possible combinations
    combinations = list(product(*combinations))
    combinations = [' '.join(combination) for combination in combinations]

    return combinations

In [6]:
intermediate.apply(lambda row: create_keywords(row), axis=1)
# intermediate['keywords'] = intermediate.apply(lambda row: create_keywords(row), axis=1)

0    [chat Arabic, chat Bengali, chat Croatian, cha...
1    [chat Arabic, chat Bengali, chat Croatian, cha...
dtype: object

In [20]:
intermediate

,intent,keyword_template,language,pma,market,account,account_id,campaign_name_template,match_type,ad_group_template,keywords
0,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",br,"subject,intent,pma,match_type","[chat Arabic, chat Bengali, chat Croatian, cha..."
1,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",xx,"subject,intent,pma,match_type","[chat Arabic, chat Bengali, chat Croatian, cha..."


In [43]:
intermediate['keywords'] = intermediate.apply(lambda row: create_keywords(row), axis=1)

/var/folders/kd/8l856r0j1gg2vg3x7txqsnqh0000gq/T/ipykernel_27476/3707104698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intermediate['keywords'] = intermediate.apply(lambda row: create_keywords(row), axis=1)


In [23]:
intermediate = intermediate.explode('keywords')

In [24]:
intermediate.dropna(subset=['keywords'], inplace=True)

In [28]:
def retrieve_dimension(row, dimension): 
    
    split_string = row['keyword_template'].split(",")

    subject_index = split_string.index(dimension)

    split_keyword = row['keywords'].split(" ")

    retrieved_word = split_keyword[subject_index]
    
    return retrieved_word

In [29]:
# Extract dimension from keywords
intermediate['subject'] = intermediate.apply(lambda row: retrieve_dimension(row, 'subject'), axis=1)

In [30]:
intermediate

,intent,keyword_template,language,pma,market,account,account_id,campaign_name_template,match_type,ad_group_template,keywords,subject
0,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",br,"subject,intent,pma,match_type",chat Arabic,Arabic
0,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",br,"subject,intent,pma,match_type",chat Bengali,Bengali
0,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",br,"subject,intent,pma,match_type",chat Croatian,Croatian
0,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",br,"subject,intent,pma,match_type",chat Czech,Czech
0,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",br,"subject,intent,pma,match_type",chat English,English
...,...,...,...,...,...,...,...,...,...,...,...,...
1,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",xx,"subject,intent,pma,match_type",chat Sanskrit,Sanskrit
1,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",xx,"subject,intent,pma,match_type",chat Math,Math
1,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",xx,"subject,intent,pma,match_type",chat Chemistry,Chemistry
1,chat,"intent,subject",eng,method,ae,AR,673-471-4088,"stu_sem_generic_web_0_,language,_,market,_xx_m...",xx,"subject,intent,pma,match_type",chat Russian,Russian


In [17]:
# Pre-calculate variables for assign_campaign_template_order
market_len = intermediate['market'].str.len()
market_ww = intermediate['market'] == 'ww'

# Assign campaign template order (vectorized)
intermediate['campaign_template_order'] = np.where((market_len == 2) & ~market_ww, 1, 2)

# Vectorized computation of campaign name
intermediate['campaign_name'] = np.where(intermediate['campaign_template_order'] == 1, 'stu_sem_generic_web_0_' + intermediate['language'] + '_' + intermediate['market'] + '_xx_multiplesub_' + intermediate['match_type'],
                                                           'stu_sem_generic_web_0_' + intermediate['language'] + '_xx_multiplesub_' + intermediate['match_type'] + '-' + intermediate['market'])

In [18]:
output_table = intermediate[['language', 'account', 'campaign_name', 'subject', 'ad_group_name', 'intent', 'keywords']]

In [19]:
end = time.time()
runtime = (end - start) / 60
print(f'The code ran in {runtime} minutes.')

The code ran in 1.7857733170191448 minutes.


In [1]:
import pandas as pd
import re
from itertools import product

In [2]:
df = pd.read_csv("dimensions_testing.csv")

In [4]:
dimension_dfs = {
    'subject': subject,
    'persona': persona,
    'other' : other,
    'medium': medium
}

In [14]:
# df['keyword_template'][0]
pattern = r'(\(.*?\)|\\\\S|\\S|\w+)'
result = re.findall(pattern, df['keyword_template'][0])
print(result)

['(intent)', '\\\\S', '(subject)', '\\\\S', '(medium)']


In [22]:
for val in result:
    if val == '(intent)':
        print(str("(" + val[1:-1] + ")"))

(intent)


In [23]:
df['keyword_template'][0]

'(intent)\\\\S(subject)\\\\S(medium)'

In [ ]:
def create_keywords(row, dimension_dfs, location):
    dimensions = re.split(r'(\(.*?\)|\\S)', str(df['keyword_template'][0]))
    language = row['language']
    market = row['market']
    combinations = []

    for dimension in dimensions:

        if dimension is None or dimension == '' or dimension == '\\':
            continue
        
        if dimension == '(location)':
            combinations.append(location['location'][location.market == market].values.tolist())
            continue

        if dimension == '(intent)':
            combinations.append([row["intent_localised"]])
            continue
        
        if dimension in dimension_dfs:
            dimension = dimension[1:-1]
            combinations.append(dimension_dfs[str('(' + dimension + ')')].loc[dimension_dfs[str('(' + dimension + ')')]['language'] == language, f'{dimension}_localised'].values.tolist())
        else:
            combinations.append([dimension])

    # Create all possible combinations
    combinations = list(product(*combinations))
    combinations = [' '.join(combination) for combination in combinations]

    return dimensions #combinations